In [ ]:
from google.colab import auth

auth.authenticate_user()

In [ ]:
%%bash
pip3 install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4393164 sha256=a53e76ffd335f28b1175ad20076dd4192c8f3ab16910ded5f312df58acda58f7
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
import numpy as np
import pandas as pd
import sklearn
from sklearn import metrics
import fasttext

In [ ]:
!gsutil cp "gs://cil_2023/train_pos_preprocessed.txt" .
!gsutil cp "gs://cil_2023/train_neg_preprocessed.txt" .

filename_train_pos = "train_pos_preprocessed.txt"
filename_train_neg = "train_neg_preprocessed.txt"

Copying gs://cil_2023/train_pos_preprocessed.txt...
\ [1 files][ 82.2 MiB/ 82.2 MiB]                                                
Operation completed over 1 objects/82.2 MiB.                                     
Copying gs://cil_2023/train_neg_preprocessed.txt...
/ [1 files][101.5 MiB/101.5 MiB]   11.5 MiB/s                                   
Operation completed over 1 objects/101.5 MiB.                                    


In [ ]:
# read dataset
dataset_pos_pd = pd.read_fwf(filename_train_pos, sep='\n', header=None, names=['text'])
dataset_neg_pd = pd.read_fwf(filename_train_neg, sep='\n', header=None, names=['text'])
dataset_pos_pd['label'] = "__label__positive"
dataset_neg_pd['label'] = "__label__negative"
dataset_pd = pd.concat([dataset_pos_pd, dataset_neg_pd])

# shuffle
dataset_pd = dataset_pd.sample(frac=1, random_state=0).reset_index(drop=True)

In [ ]:
N = len(dataset_pd)
train_size = int(0.9 * N)
dataset_train = dataset_pd[:train_size]
dataset_val = dataset_pd[train_size:]

In [ ]:
# fasttext
np.savetxt(r'data.train.txt', dataset_train.values, fmt='%s')
np.savetxt(r'data.val.txt', dataset_val.values, fmt='%s')

model = fasttext.train_supervised('data.train.txt')
_, accuracy, _ = model.test('data.val.txt')
print("Accuracy: {}".format(accuracy))

Accuracy: 0.836676
